In [1]:
import numpy as np
import IPython.display as ipd  
import matplotlib.pyplot as plt
import networkx as nx

fs = 48000
pi = np.pi

In [2]:
pick = lambda n : lambda x : x[0][:,n][:,np.newaxis]
const = lambda n : lambda x : n
mult = lambda x : x[0] * x[1]
plus = lambda x : x[0] + x[1]
minus = lambda x : x[0] - x[1]
divide = lambda x : x[0] / x[1]
wav = lambda x : np.sin(2*pi*x[0])
exp = lambda x : np.exp(x[0])
merge = lambda x : np.sum(x[0],axis=0)
H = lambda x : np.heaviside(x[0])
idy = lambda x : x[0]

In [3]:
def compute(A,ops):
    
    roots = np.where(np.logical_not(np.any(A,axis=1)))[0] 
    B = np.stack((np.identity(A.shape[0]),A))
    
    while np.any(B[-1]):
        B = np.vstack((B,(B[1] @ B[-1])[np.newaxis,:]))
    
    path = np.sum(B[:,:,roots],axis=-1)
    path = np.heaviside(path,0).astype(int)
    order = path.shape[0] - np.argmax(path[::-1], axis=0) - 1
    
    values = [None for op in ops]
    
    for n in range(max(order) + 1):
        inds = np.where(order == n)[0]
        for k in inds:
            input_inds = np.where(A[k])[0]
            input_vals = [values[l] for l in input_inds]
            values[k] = ops[k](input_vals)
                       
    return values[np.argmax(order)]

In [4]:
y = lambda y : np.array([[1,-1,1200],[1,-2,1500]])
x = lambda x : np.arange(0,4,1/fs)

ops = [merge,mult,exp,wav,plus,pick(0),mult,pick(1),pick(2),x,y]

A = np.array(
     # >  *  e  w  +  0  *  1  2  x  y
    [[ 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], # >
     [ 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0], # *
     [ 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0], # e
     [ 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], # w
     [ 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0], # +
     [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], # 0
     [ 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0], # *
     [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], # 1
     [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], # 2
     [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], # x
     [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])# y


In [5]:
test = compute(A,ops)
ipd.display(ipd.Audio(test, rate=fs))  